# USAA Y-9C Financial Data Analysis

This notebook provides interactive analysis of USAA's Y-9C regulatory filings.

## Setup
First, ensure you have downloaded and loaded the data by running:
```bash
python run_scraper.py --init
```

In [ ]:
import sys
sys.path.insert(0, '.')

import pandas as pd
import sqlite3
from pathlib import Path

from y9c_config import USAA_HOLDING_COMPANY_RSSD, get_all_mdrm_codes
from y9c_database import DB_PATH, get_connection, get_balance_sheet, get_income_statement, get_all_periods

print(f"Database: {DB_PATH}")
print(f"USAA RSSD ID: {USAA_HOLDING_COMPANY_RSSD}")

## Load Data into Pandas DataFrames

In [ ]:
# Connect to database and load all financial data
conn = sqlite3.connect(DB_PATH)

# Load all financial data with account names
query = """
SELECT 
    fd.report_date,
    fd.year,
    fd.quarter,
    fd.mdrm_code,
    ad.account_name,
    ad.statement_type,
    ad.category,
    fd.value
FROM financial_data fd
JOIN account_definitions ad ON fd.mdrm_code = ad.mdrm_code
WHERE fd.rssd_id = ?
ORDER BY fd.report_date, ad.statement_type, ad.category
"""

df = pd.read_sql_query(query, conn, params=[USAA_HOLDING_COMPANY_RSSD])
df['report_date'] = pd.to_datetime(df['report_date'])

conn.close()

print(f"Loaded {len(df):,} records")
print(f"Date range: {df['report_date'].min()} to {df['report_date'].max()}")
df.head()

## Balance Sheet Analysis

In [ ]:
# Filter to balance sheet items
bs_df = df[df['statement_type'] == 'balance_sheet'].copy()

# Pivot to get time series by account
bs_pivot = bs_df.pivot_table(
    index='report_date',
    columns='account_name',
    values='value',
    aggfunc='first'
)

print(f"Balance sheet accounts: {len(bs_pivot.columns)}")
bs_pivot.tail()

In [ ]:
# Plot Total Assets over time
if 'Total assets' in bs_pivot.columns:
    ax = bs_pivot['Total assets'].plot(
        figsize=(12, 6),
        title='USAA Total Assets ($ thousands)',
        marker='o',
        markersize=3
    )
    ax.set_ylabel('$ thousands')
    ax.set_xlabel('Date')
else:
    print("Total assets not found. Available columns:")
    print(bs_pivot.columns.tolist())

## Income Statement Analysis

In [ ]:
# Filter to income statement items
inc_df = df[df['statement_type'] == 'income_statement'].copy()

# Pivot to get time series by account
inc_pivot = inc_df.pivot_table(
    index='report_date',
    columns='account_name',
    values='value',
    aggfunc='first'
)

print(f"Income statement accounts: {len(inc_pivot.columns)}")
inc_pivot.tail()

In [ ]:
# Plot Net Interest Income over time
income_cols = ['Net interest income', 'Total interest income', 'Total interest expense']
available_cols = [c for c in income_cols if c in inc_pivot.columns]

if available_cols:
    ax = inc_pivot[available_cols].plot(
        figsize=(12, 6),
        title='USAA Interest Income/Expense ($ thousands)',
        marker='o',
        markersize=3
    )
    ax.set_ylabel('$ thousands')
    ax.set_xlabel('Date')
    ax.legend(loc='best')
else:
    print("Interest income columns not found. Available columns:")
    print(inc_pivot.columns.tolist())

## Key Financial Ratios

In [ ]:
# Calculate key ratios
ratios = pd.DataFrame(index=bs_pivot.index)

# Equity to Assets ratio
if 'Total assets' in bs_pivot.columns and 'Total equity capital' in bs_pivot.columns:
    ratios['Equity/Assets'] = bs_pivot['Total equity capital'] / bs_pivot['Total assets']

# Net Interest Margin (approximate - uses year-to-date values)
if 'Net interest income' in inc_pivot.columns and 'Total assets' in bs_pivot.columns:
    # Merge income and balance sheet data
    ratios['NIM (approx)'] = (inc_pivot['Net interest income'] * 4) / bs_pivot['Total assets']

if len(ratios.columns) > 0:
    ratios.plot(figsize=(12, 6), title='Key Financial Ratios', subplots=True, layout=(1, -1))
else:
    print("Could not calculate ratios - missing required columns")

## Export Latest Financial Statements

In [ ]:
# Get the most recent period
latest_date = df['report_date'].max()
latest_year = df[df['report_date'] == latest_date]['year'].iloc[0]
latest_quarter = df[df['report_date'] == latest_date]['quarter'].iloc[0]

print(f"Most recent data: {latest_date.strftime('%Y-%m-%d')} (Q{latest_quarter} {latest_year})")

# Balance Sheet
print("\n" + "="*60)
print("BALANCE SHEET")
print("="*60)

latest_bs = df[(df['report_date'] == latest_date) & (df['statement_type'] == 'balance_sheet')]
for category in ['assets', 'liabilities', 'equity']:
    cat_data = latest_bs[latest_bs['category'] == category]
    if len(cat_data) > 0:
        print(f"\n{category.upper()}:")
        for _, row in cat_data.iterrows():
            if pd.notna(row['value']):
                print(f"  {row['account_name']}: ${row['value']:,.0f}")

In [ ]:
# Income Statement
print("\n" + "="*60)
print("INCOME STATEMENT")
print("="*60)

latest_inc = df[(df['report_date'] == latest_date) & (df['statement_type'] == 'income_statement')]
for category in ['interest_income', 'interest_expense', 'net_interest_income', 
                 'provision', 'noninterest_income', 'noninterest_expense', 'income']:
    cat_data = latest_inc[latest_inc['category'] == category]
    if len(cat_data) > 0:
        print(f"\n{category.upper().replace('_', ' ')}:")
        for _, row in cat_data.iterrows():
            if pd.notna(row['value']):
                print(f"  {row['account_name']}: ${row['value']:,.0f}")

## Data Quality Check

In [ ]:
# Check data coverage by quarter
coverage = df.groupby(['year', 'quarter']).agg({
    'mdrm_code': 'nunique',
    'value': 'count'
}).rename(columns={'mdrm_code': 'unique_codes', 'value': 'total_records'})

coverage